In [250]:
#optional
#fixes errors with importing spacy on mac OS systems on python v3.12.6
import ssl
import nltk

ssl._create_default_https_context = ssl._create_unverified_context
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/umar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In this file we are going to perform sentiment analysis on review data taken from an imdb dataset. I will attempt to use nlp to form a classification models to predict the number of positive/negative reviews.

I will include samples to demonstrate what certain areas of code are doing, # indicates a line of code, ## indicates a line explaining the code


In [251]:


#import necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import nltk
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from nltk.corpus import stopwords
stopword=set(stopwords.words('english'))

Exploratory Data Analysis(EDA)

In [252]:
#view the data
imdb=pd.read_csv('/Users/umar/Documents/datasets/IMDB Dataset.csv')
print(imdb)
imdb.head()
imdb.describe()

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [253]:
imdb['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [254]:
##remove stopwords for the dataset, stopwords are common words such as 'an' or 'are' which add little information about sentiment to the text
#create list of stopwords in desired language

#specify tokeniser
tokeniser=ToktokTokenizer()

#define our function to remove stopwords
def remove_stopwords(rev, is_lower_case=False):
    tokens = tokeniser.tokenize(rev) #tokenise our text using pre built tokeniser
    tokens=[token.strip() for token in tokens] #remove whitespaces
    if is_lower_case:
        filter_tokens= [token for token in tokens if token not in stopwords] # we text each token against the stopword list
    else:
        filter_tokens = [token for token in tokens if token.lower() not in stopword] #if token is not lowercase we test the lowercase version against the stopword list
    filtered_reviews = ' '.join(filter_tokens) # we join this futher filtered text into a list after it is parsed
    return filtered_reviews

#test corpus as an example
#text=  "I remember as a child, and as a young budding naturalist, spending all my time observing and testing the world around me"
#text_stopwords=remove_stopwords(text)
#print('removed stopwords:', text_stopwords)

imdb['review']=imdb['review'].apply(remove_stopwords)

In [255]:
#splitting the dataset for training/testing train/test 
#here I am employing the common 80:20 rule for testing
train_sentiment=imdb['sentiment'][:40000]
train_reviews=imdb['review'][:40000]
test_sentiment=imdb['sentiment'][40000:]
test_reviews=imdb['review'][40000:]
#we should expect 0 to ensure same length of datasets
print(len(train_reviews)-len(train_sentiment))
print(len(test_reviews)-len(test_sentiment))

0
0


In [256]:
#For a key inital step in the NLP pipeline we need to tokenise the dataset
tokeniser=ToktokTokenizer()
#reduce noise and increase effiency from removing stopwords
stopwords=nltk.corpus.stopwords.words('english')

In [257]:
#strip additonal noise
#remove html
def html(rev):
    soup = BeautifulSoup(rev, "html.parser")
    return soup.get_text()

import re

def rem_sq_brkets(rev):
    return re.sub(r'\[[^]]*\]', '', rev) 


def denoise(rev):
    rev = html(rev)
    rev = rem_sq_brkets(rev)
    return rev

imdb['review']=imdb['review'].apply(denoise)

/var/folders/pz/ppj3c3y97wb28xcmb96ybk6r0000gp/T/ipykernel_10392/3175857043.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(rev, "html.parser")


In [258]:
#futher denoising reviews column
#use regex to target characters that arent in a-z,0-9, uppercase letters or a space
def rem_special_characters(rev, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    rev=re.sub(pattern,'',rev)
    return rev

imdb['review']=imdb['review'].apply(rem_special_characters)

In [259]:
#Reduce the dimenionsality of our dataset
#We reduce words to their original formatting e.g. carrying -> carry
#in this implementation I will be using the snowball stemmer for improved flexibility and language support
#this will allow the code to become more flexible and take in a bigger dataset of multuple language reviews

from nltk.stem import SnowballStemmer

##############Run this sample code to see the stemmer 
##retrieve desired stemmer
#snowball=SnowballStemmer("english")
## Examples
#samples= ["carrying", "carry","runs", "easily", "quickly"]
##stem our sample words by looping over text
#for sample in samples:
#   base=snowball.stem(sample)
#    print(f"{sample} -> {base}")
######################################################

#stem our imdb text
def stemmer(rev):
    snow=SnowballStemmer('english')
    rev = ' '.join([snow.stem(word) for word in rev.split()])
    return rev

############function test using sample text
#text=  "I remember as a child, and as a young budding naturalist, spending all my time observing and testing the world around me"
##stem_text=stemmer(text)
#print("stemmed text:", stem_text)

imdb['review']=imdb['review'].apply(stemmer)
imdb['review'].head()

0    one review mention watch 1 oz episod hook righ...
1    wonder littl product the film techniqu unassum...
2    thought wonder way spend time hot summer weeke...
3    basic famili littl boy jake think zombi closet...
4    petter mattei love time money visual stun film...
Name: review, dtype: object

In [260]:
##remove stopwords for the dataset, stopwords are common words such as 'an' or 'are' which add little information about sentiment to the text
#create list of stopwords in desired language

#specify tokeniser
tokeniser=ToktokTokenizer()

#define our function to remove stopwords
def remove_stopwords(rev, is_lower_case=False):
    tokens = tokeniser.tokenize(rev) #tokenise our text using pre built tokeniser
    tokens=[token.strip() for token in tokens] #remove whitespaces
    if is_lower_case:
        filter_tokens= [token for token in tokens if token not in stopwords] # we text each token against the stopword list
    else:
        filter_tokens = [token for token in tokens if token.lower() not in stopword] #if token is not lowercase we test the lowercase version against the stopword list
    filtered_reviews = ' '.join(filter_tokens) # we join this futher filtered text into a list after it is parsed
    return filtered_reviews

#test corpus as an example
#text=  "I remember as a child, and as a young budding naturalist, spending all my time observing and testing the world around me"
#text_stopwords=remove_stopwords(text)
#print('removed stopwords:', text_stopwords)

imdb['review']=imdb['review'].apply(remove_stopwords)

In [261]:
#filtered reviews
filtered_train_rev=imdb['review'][:40000]
print(filtered_train_rev[0]) # tests
filtered_test_rev=imdb['review'][40000:]
#print(filtered_test_rev[40001])

one review mention watch 1 oz episod hook right exact happen meth first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use wordit call oz nicknam given oswald maximum secur state penitentari focus main emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far awayi would say main appeal show due fact goe show dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz mess around first episod ever saw struck nasti surreal say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view get touch darker side


In [262]:
#use a bag od words model to covert the corpus of text in a ml friendly format. This format is centered around word frequency. 
#Here we will vectorise the word corpus using the WordVectoriser from the scikit learn package we loaded 

#An example of how it works
#initialise our count vectoriser
cv=CountVectorizer(min_df=0.0,max_df=1.0, binary=False, ngram_range=(1,3))
##min_df=minimum document frequency a word should appear in to be included in vocabulary
##max_df=minimum document frequency a word should appear in to be included in vocabulary
##binary=False indicates that we are condidering both the prescence(Binary=True) and the frequncy of words in the corpus
##ngram_range indicates that the vectoriser will consider combinations of up to 3 words from the text to allow for more consideration of contextual information byt increasing the dimensionality of our model affected efficiency
sample_doc = [
    "The cat sat on the mat.",
    "The dog barked at the cat.",
    "Cats and dogs are common pets."
]
#our sample bag of words
bow_sample=cv.fit_transform(sample_doc)
print(bow_sample.toarray())
print(cv.get_feature_names_out())

[[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 1 2 1
  1 0 0 1]
 [0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 2 1
  0 1 1 0]
 [1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0
  0 0 0 0]]
['and' 'and dogs' 'and dogs are' 'are' 'are common' 'are common pets' 'at'
 'at the' 'at the cat' 'barked' 'barked at' 'barked at the' 'cat'
 'cat sat' 'cat sat on' 'cats' 'cats and' 'cats and dogs' 'common'
 'common pets' 'dog' 'dog barked' 'dog barked at' 'dogs' 'dogs are'
 'dogs are common' 'mat' 'on' 'on the' 'on the mat' 'pets' 'sat' 'sat on'
 'sat on the' 'the' 'the cat' 'the cat sat' 'the dog' 'the dog barked'
 'the mat']


In [263]:
cv=CountVectorizer(min_df=0.0,max_df=1.0, binary=False, ngram_range=(1,3))
#fit to train reviews
cv_train_rev=cv.fit_transform(filtered_train_rev)
#transform test reviews
cv_test_rev=cv.transform(filtered_test_rev)
#View outcomes
print('Bag of words trained reviews:', cv_train_rev.shape)
print('Bag of words test reviews:', cv_test_rev.shape)

Bag of words trained reviews: (40000, 6687752)
Bag of words test reviews: (10000, 6687752)


In [264]:
#Our next model is a Term Frequency-Inverse Document Frequency model (TFIDF) model.
#Term frequency: Evaluates how frequency (number of times term appears/total number of terms in the text) a word appears in a corpus of text
#Inverse document theory: log(total number of doc/number of documents containing the word)
#TF-IDF=TFxIDF
#A higher TF-IDF score indicates that a word is more important across a corpus of text, however it ignores context e.g. apple the fruit or apple the company

#A sample to understand implementation
#Initilise vectoriser
tfidf=TfidfVectorizer(min_df=0.0,max_df=1.0,use_idf=True,ngram_range=(1,3)) #set 1-3 to keep comparisons between models
sample_doc = [
    "The cat sat on the mat.",
    "The dog barked at the cat.",
    "Cats and dogs are common pets."
]
tfidf_sample=tfidf.fit_transform(sample_doc)
print(tfidf_sample.toarray())
print(tfidf.get_feature_names_out())

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.19992804 0.26288135 0.26288135 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.26288135 0.26288135 0.26288135 0.26288135
  0.         0.26288135 0.26288135 0.26288135 0.39985609 0.19992804
  0.26288135 0.         0.         0.26288135]
 [0.         0.         0.         0.         0.         0.
  0.26288135 0.26288135 0.26288135 0.26288135 0.26288135 0.26288135
  0.19992804 0.         0.         0.         0.         0.
  0.         0.         0.26288135 0.26288135 0.26288135 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.39985609 0.19992804
  0.         0.26288135 0.26288135 0.        ]
 [0.25819889 0.25819889 0.25819889 0.25819889 0.25819889 0.25819889
  0.         0.         0.         0.         0.         0.
  0.         0.         0.

In [265]:
tfidf=TfidfVectorizer(min_df=0.0, max_df=1.0,use_idf=True,ngram_range=(1,3))

tfidf_train_rev=tfidf.fit_transform(filtered_train_rev)
tfidf_test_rev=tfidf.transform(filtered_test_rev)
print('tfidf training reviews:', tfidf_train_rev.shape)
print('tfidf testing reviews:', tfidf_test_rev.shape)


tfidf training reviews: (40000, 6687752)
tfidf testing reviews: (10000, 6687752)


In [266]:
#binarise our sentiment data to trasnform categorical labels into a format machine learning can understand
#A sample implementation
data={'review':['great movie!', 'terrible movie'], 'sentiment':['positive', 'negative']}
sample_data=pd.DataFrame(data)
print(sample_data.head())
#initialise label binariser
lb=LabelBinarizer()#transform our sentiment data into binary format
binarise_data=lb.fit_transform(sample_data['sentiment'])
print(binarise_data)

           review sentiment
0    great movie!  positive
1  terrible movie  negative
[[1]
 [0]]


In [267]:
lb=LabelBinarizer()
sentiment_trans=lb.fit_transform(imdb['sentiment'])
print(sentiment_trans)
print(sentiment_trans.shape)



[[1]
 [1]
 [1]
 ...
 [0]
 [0]
 [0]]
(50000, 1)


In [268]:
#split into train/test sets
train_sentiment=sentiment_trans[:40000]
test_sentiment=sentiment_trans[40000:]
print(train_sentiment.shape)
print(test_sentiment.shape)

(40000, 1)
(10000, 1)


Model the dataset using logistic regression

In [269]:
#specify the model
lr=LogisticRegression(penalty='l2',max_iter=500,C=1, random_state=123)
#The penalty is the approach we are taking, in this case we are using a ridge regression model which has a heavy penalty on large weights leading to reduction in overfitting issues
#The iteration specification gives a limit to how many parameters the model will attempt to converge
#the C parameter is the regularisation parameter which defines the strength of the penalty. It is a positive number with a lower number defining a larger penalty and is helpful to prevent overfitting.
#I define a random state for reproducability of results

#apply to our bag of words approach
lr_cv=lr.fit(cv_train_rev,train_sentiment)
print(lr_cv)

#fit model to tfidf approach
lr_tfidf=lr.fit(tfidf_train_rev, train_sentiment)
print(lr_tfidf)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1, max_iter=500, random_state=123)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1, max_iter=500, random_state=123)


In [270]:
#Use our models to predict sentiments
#Bag of words model
lr_cv_pred=lr.predict(cv_test_rev)
print(lr_cv_pred)
#TFIDF model
lr_tfidf_pred=lr.predict(tfidf_test_rev)
print(lr_tfidf_pred)

[0 0 0 ... 1 0 0]
[0 0 0 ... 1 0 0]


In [271]:
#evaluate model accuracy
#Bag of words model
lr_cv_accurary_score=accuracy_score(test_sentiment, lr_cv_pred)
print('logistic regression bag of words accuracy score:',lr_cv_accurary_score)
#TFIDF model
lr_tfidf_accurary_score=accuracy_score(test_sentiment, lr_tfidf_pred)
print('logistic regression TFIDF accuracy score:',lr_tfidf_accurary_score)

logistic regression bag of words accuracy score: 0.8592
logistic regression TFIDF accuracy score: 0.8854


In [272]:
#Print a classification report
#Bag of words model
lr_cv_report=classification_report(test_sentiment,lr_cv_pred,target_names=['Positive','Negative'])
print(lr_cv_report)
#TFIDF model
lr_tfidf_report=classification_report(test_sentiment,lr_tfidf_pred, target_names=['Positive','Negative'])
print(lr_tfidf_report)

              precision    recall  f1-score   support

    Positive       0.87      0.84      0.86      4993
    Negative       0.85      0.88      0.86      5007

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000

              precision    recall  f1-score   support

    Positive       0.90      0.87      0.88      4993
    Negative       0.88      0.90      0.89      5007

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [273]:
#Confusion Matrix
#bag of words approach
cv_cm=confusion_matrix(test_sentiment,lr_cv_pred,labels=[1,0])
print(cv_cm)
tfidf_cm=confusion_matrix(test_sentiment,lr_tfidf_pred,labels=[1,0])
print(tfidf_cm)
#####In the confusion matrix:
#rows represent the actual outcome
#columns represent the predicted outcome
#True/False Positive(tp/fp) or True/False Negative(tn/fn)
#key metrics

[[4384  623]
 [ 785 4208]]
[[4498  509]
 [ 637 4356]]


Stochastic Gradient Descent model

In [274]:
#training model
sgd=SGDClassifier(loss='hinge',max_iter=500,random_state=123)
#fitting model to bag of words data
sgd_cv=sgd.fit(cv_train_rev, train_sentiment)
print(sgd_cv)
#fitting model to tfidf approach data 
sgd_tfidf=sgd.fit(tfidf_train_rev,train_sentiment)
print(sgd_tfidf)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier(max_iter=500, random_state=123)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier(max_iter=500, random_state=123)


In [275]:
#Evaluate SGD model performance on test data
sgd_cv_pred=sgd.predict(cv_test_rev)
print(sgd_cv_pred)
sgd_tfidf_pred=sgd.predict(tfidf_test_rev)
print(sgd_tfidf_pred)


[0 0 0 ... 1 0 0]
[0 0 0 ... 1 0 0]


In [276]:
#Model Accuracy
sgd_cv_accuracy=accuracy_score(test_sentiment, sgd_cv_pred)
print('SGD Accuracy score for Bag of words:',sgd_cv_accuracy)
sgd_tfidf_accuracy=accuracy_score(test_sentiment, sgd_tfidf_pred)
print('SGD Accuracy score for TDIDF:',sgd_tfidf_accuracy)

SGD Accuracy score for Bag of words: 0.8605
SGD Accuracy score for TDIDF: 0.8834


In [277]:
#produce a classification report for the SGD models
#Bag of Words approach
sgd_cv_report=classification_report(test_sentiment, sgd_cv_pred, target_names=['Positive','Negative'])
print(sgd_cv_report)
#TFIDF approach
sgd_tfidf_report=classification_report(test_sentiment,sgd_tfidf_pred, target_names=['Positive','Negative'])
print(sgd_tfidf_report)

              precision    recall  f1-score   support

    Positive       0.85      0.87      0.86      4993
    Negative       0.87      0.85      0.86      5007

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000

              precision    recall  f1-score   support

    Positive       0.90      0.86      0.88      4993
    Negative       0.87      0.90      0.89      5007

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



In [278]:
#Plot the confusion Matrices
#Bag of Words
sgd_cv_matrix=confusion_matrix(test_sentiment,sgd_cv_pred,labels=[1,0])
print(sgd_cv_matrix)
#TFIDF approacg
sgd_tfidf_matrix=confusion_matrix(test_sentiment,sgd_tfidf_pred,labels=[1,0])
print(sgd_tfidf_matrix)

[[4264  743]
 [ 652 4341]]
[[4520  487]
 [ 679 4314]]


Naive Bayes Model

In [279]:
#Initialise our Model
mnb=MultinomialNB()
#Training for bag of words
mnb_cv=mnb.fit(cv_train_rev, train_sentiment)
print(mnb_cv)
#Training for TDIDF approach
mnb_tfidf=mnb.fit(tfidf_train_rev,train_sentiment)
print(mnb_tfidf)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB()
MultinomialNB()


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [282]:
#Use Model to make predictions on test data
#Bag of Words approach
mnb_cv_pred=mnb.predict(cv_test_rev)
print(mnb_cv_pred)
#TFIDF approach
mnb_tfidf_pred=mnb.predict(tfidf_test_rev)
print(mnb_tfidf_pred)

[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


In [283]:
#test model accuracy
#bag of words approach
mnb_cv_accuracy=accuracy_score(test_sentiment,mnb_cv_pred)
print('MNB bag of words prediction score:', mnb_cv_accuracy)
#TFIDF approach
mnb_tfidf_accuracy=accuracy_score(test_sentiment,mnb_tfidf_pred)
print('MNB TFIDF prediction score:', mnb_tfidf_accuracy)

MNB bag of words prediction score: 0.8797
MNB TFIDF prediction score: 0.8874


In [286]:
#produce classification reports
#Bag of Words approach
mnb_cv_report=classification_report(test_sentiment,mnb_cv_pred, target_names=['Positive,', 'Negative'])
print(mnb_cv_report)
#TFIDF approach
mnb_tfidf_report=classification_report(test_sentiment,mnb_tfidf_pred, target_names=['Positive,', 'Negative'])
print(mnb_tfidf_report)

              precision    recall  f1-score   support

   Positive,       0.87      0.90      0.88      4993
    Negative       0.89      0.86      0.88      5007

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000

              precision    recall  f1-score   support

   Positive,       0.89      0.89      0.89      4993
    Negative       0.89      0.89      0.89      5007

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [287]:
#Confusion Matrices
#Bag of Words approach
mnb_cv=confusion_matrix(test_sentiment, mnb_cv_pred, labels=[1,0])
print(mnb_cv)
#TFIDF Approach
mnb_tfidf=confusion_matrix(test_sentiment, mnb_tfidf_pred, labels=[1,0])
print(mnb_tfidf)

[[4326  681]
 [ 522 4471]]
[[4450  557]
 [ 569 4424]]


It can be seen that the SGD model performed worse than our other models.

I believe that gains in model accuracy would be better realised by pursing more advanced forms of preprocessing the data such as RoBERTa (Robustly optimized BERT approach) which would allow us to better
capture the importance of context in the data